In [1]:
import Utils
import pandas as pd
import numpy as np

In [2]:
def bs_from_row(row):
    """
    Convert DataFrame row to numeric parameters and compute Greeks.
    Automatically converts DaysToMaturity / DayPerYear to T (in years).
    """
    try:
        S = float(row['Underlying'])
        K = float(row['Strike'])
        T = float(row['DaysToMaturity']) / float(row['DayPerYear'])
        r = float(row['RiskFreeRate'])
        q = float(row['DividendRate'])
        sigma = float(row['ImpliedVol'])
        option_type = str(row['Option Type']).lower().strip()
    except Exception as e:
        raise ValueError(f"Invalid data in row {row.name}: {e}")

    return pd.Series(Utils.bs_european_greeks(S, K, T, r, q, sigma, option_type))


def bs_dataframe(df):
    """
    Apply BSM pricing and Greeks to an entire DataFrame.
    Includes automatic numeric conversion for safety.
    """
    df = df.copy()

    # Automatically convert numeric columns to float
    numeric_cols = [
        'Underlying', 'Strike', 'DaysToMaturity', 'DayPerYear',
        'RiskFreeRate', 'DividendRate', 'ImpliedVol'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Detect non-numeric entries (debug help)
    non_numeric = df[numeric_cols].select_dtypes(exclude='number')
    if not non_numeric.empty:
        print("Warning: Some numeric columns contain non-numeric values.")

    # Apply row-by-row computation
    result = df.apply(bs_from_row, axis=1).round(6)

    return result

def american_from_row_continuous_div(row, steps=200):
    """
    Extracts parameters from a single DataFrame row and computes American option Greeks.

    Parameters
    ----------
    row : pandas.Series
        A single row from the input DataFrame.
    steps : int
        Number of time steps for the binomial tree.

    Returns
    -------
    pandas.Series : A series containing the option Price and Greeks.
    """
    
    # Extract parameters, ensuring correct type conversion
    S = float(row['Underlying'])
    K = float(row['Strike'])
    # Convert DaysToMaturity to Time to Maturity (in years)
    T = float(row['DaysToMaturity']) / float(row['DayPerYear'])
    r = float(row['RiskFreeRate'])
    q = float(row['DividendRate'])
    sigma = float(row['ImpliedVol'])
    # Clean the option type string
    option_type = str(row['Option Type']).lower().strip() 

    # Call the core function that calculates Price and Greeks
    return pd.Series(Utils.american_binomial_with_greeks(S, K, T, r, q, sigma, steps, option_type))


def american_dataframe_continuous(df, steps=200):
    """
    Computes American option price and Greeks for all rows in a DataFrame.

    Parameters
    ----------
    df : pandas.DataFrame
        Input data containing option parameters.
    steps : int
        Number of time steps for the binomial tree.

    Returns
    -------
    pandas.DataFrame : A new DataFrame with calculated Price and Greeks.
    """
    df = df.copy()

    # Identify and convert numeric columns to float, coercing errors to NaN
    numeric_cols = [
        'Underlying', 'Strike', 'DaysToMaturity', 'DayPerYear',
        'RiskFreeRate', 'DividendRate', 'ImpliedVol'
    ]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Apply the calculation function row-by-row (axis=1)
    # Note: Higher steps (e.g., 400-500) are generally recommended for stability.
    result = df.apply(lambda row: american_from_row_continuous_div(row, steps), axis=1).round(6)

    return result

def american_price_from_row_discrete(row):
    """
    Convert DataFrame row with discrete dividend strings to numeric parameters,
    and compute the American option price using bt_american_discrete_div.
    """
    
    S = float(row['Underlying'])
    K = float(row['Strike'])
    DaysToMaturity = float(row['DaysToMaturity'])
    DayPerYear = float(row['DayPerYear'])
    T = DaysToMaturity / DayPerYear
    r = float(row['RiskFreeRate'])
    sigma = float(row['ImpliedVol'])
    option_type = str(row['Option Type']).lower().strip()
    
    steps = int(DaysToMaturity)
    
    try:
        amts_str = str(row['DividendAmts']).replace(' ', '')
        divAmts = [float(x) for x in amts_str.split(',') if x]
        
        dates_str = str(row['DividendDates']).replace(' ', '')
        divDates_days = [float(x) for x in dates_str.split(',') if x]
        
        divTimes = [int(x) for x in divDates_days]
        
    except Exception as e:
        print(f"Error parsing dividend data for row {row['ID']}: {e}")
        return pd.Series({'Price': np.nan})

    divAmts_filtered = []
    divTimes_filtered = []
    for amt, time in zip(divAmts, divTimes):
        if 0 < time <= steps:
            divAmts_filtered.append(amt)
            divTimes_filtered.append(time)

    price = Utils.bt_american_discrete_div(
        S, K, T, r, divAmts_filtered, divTimes_filtered, sigma, steps, option_type
    )

    return pd.Series({'Price': price})

def american_dataframe_discrete(df):
    """
    Computes American option price (Price only) for all rows with discrete dividends.
    """
    df = df.copy()

    numeric_cols = [
        'Underlying', 'Strike', 'DaysToMaturity', 'DayPerYear',
        'RiskFreeRate', 'ImpliedVol'
    ]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    result = df.apply(american_price_from_row_discrete, axis=1).round(6)

    return result

In [3]:
## Test12.1
data = pd.read_csv("../testfiles/data/test12_1.csv").dropna()
bs_dataframe(data)

,Price,Delta,Gamma,Vega,Rho,Theta
0,3.260824,0.547872,0.053506,14.659079,7.058414,-13.019817
1,2.646281,-0.452128,0.053506,14.659079,-6.556032,-8.547471
2,22.043329,0.685508,0.008115,35.571438,50.967099,-7.213608
3,20.449083,-0.470751,0.009310,40.812329,-73.999110,-5.351164


In [4]:
## Test12.2
american_dataframe_continuous(data)

,Price,Delta,Gamma,Vega,Rho,Theta
0,3.257134,0.547813,0.055942,14.641168,7.058559,13.060835
1,2.691417,-0.463249,0.057471,14.603766,-5.246725,8.989991
2,22.055215,0.672456,0.000025,36.019335,50.217826,7.305812
3,21.029189,-0.485821,0.002770,40.883540,-53.738031,5.982709


In [5]:
## Test12.3
data = pd.read_csv("../testfiles/data/test12_3.csv").dropna()
american_dataframe_discrete(data)

,Price
0,14.497036
1,11.775958
